## Import libraries

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import time
import warnings
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn import datasets, linear_model
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras import regularizers
import keras.backend as K
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt
from keras.callbacks import ModelCheckpoint

plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

# prevent tensorflow from allocating the entire GPU memory at once
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
#warnings.filterwarnings("ignore") #Hide messy Numpy warnings

%autosave 60

Using TensorFlow backend.


Autosaving every 60 seconds


## Load data

In [2]:
def load_data(filename, num_lags, bootstrap_size):
    assert bootstrap_size > num_lags
    
    # read data from file
    f = open(filename)
    series = []
    removed_seasonality = [] 
    removed_std = []
    missings = []
    for line in f:
        splt = line.split(",")
        series.append(float(splt[0]))
        removed_seasonality.append(float(splt[1]))
        removed_std.append(float(splt[2]))
        missings.append(int(splt[3]))
    series = np.array(series)
    removed_seasonality = np.array(removed_seasonality)
    removed_std = np.array(removed_std)
    missings = np.array(missings)
    f.close()

    # generate lags
    X = []
    for i in range(bootstrap_size, len(series)):
        X.append(series[i-num_lags:i])
    X = np.array(X)
    
    y = series[bootstrap_size:]
    removed_seasonality = removed_seasonality[bootstrap_size:]
    removed_std = removed_std[bootstrap_size:]
    missings = missings[bootstrap_size:]
    assert X.shape[0] == y.shape[0]

    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, y, removed_seasonality, removed_std, missings

In [3]:
# read data from different places
NUM_LAGS = 50
bootstrap_size = 12*24*1
ids = []
removed_seasonality_all = []
removed_std_all = []
missings_all = []
X_all = []
y_all = []
for fname in os.listdir('norrecampus'):
    print "reading data for place id:", fname[-6:-4]
    X, y, removed_seasonality, removed_std, missings = load_data('norrecampus/'+fname, NUM_LAGS, bootstrap_size)
    ids.append(fname[-6:-4])
    X_all.append(X)
    y_all.append(y)
    removed_seasonality_all.append(removed_seasonality)
    removed_std_all.append(removed_std)
    missings_all.append(missings)
X_all = np.array(X_all)
y_all = np.array(y_all)
print X_all.shape
print y_all.shape
n_places = len(ids)
n_instances = X_all.shape[1]
print "n_instances:", n_instances
print "n_places:", n_places
print "n_lags:", NUM_LAGS

reading data for place id: s8
reading data for place id: _k
reading data for place id: QE
reading data for place id: 5U
reading data for place id: LQ
reading data for place id: vU
reading data for place id: aM
reading data for place id: Gc
reading data for place id: kI
(9, 51552, 50, 1)
(9, 51552)
n_instances: 51552
n_places: 9
n_lags: 50


In [4]:
# reshape data
X = np.swapaxes(X_all, 0, 1)
X = np.swapaxes(X, 1, 2)
X = X[:,:,:,:,np.newaxis]
print X.shape
y = np.swapaxes(y_all, 0, 1)
y = y[:,np.newaxis,:,np.newaxis,np.newaxis]
print y.shape

STEPS_AHEAD = 1
X = X[:,:NUM_LAGS-STEPS_AHEAD+1,:,:]
print X.shape

(51552, 50, 9, 1, 1)
(51552, 1, 9, 1, 1)
(51552, 50, 9, 1, 1)


## Train/test split

In [5]:
n_train = 12*24*89
n_test = n_train + 12*24*90
X_train = X[:n_train,:]
y_train = y[:n_train]
X_test = X[n_train:n_test,:]
y_test = y[n_train:n_test]
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

(25632, 50, 9, 1, 1)
(25632, 1, 9, 1, 1)
(25920, 50, 9, 1, 1)
(25920, 1, 9, 1, 1)


## Evaluation functions

In [6]:
def compute_error(trues, predicted):
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    mse = np.mean((predicted - trues)**2)
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    r2 = max(0, 1 - np.sum((trues-predicted)**2) / np.sum((trues - np.mean(trues))**2))
    return corr, mae, mse, rae, rmse, r2

In [7]:
def compute_error_filtered(trues, predicted, filt):
    trues = trues[filt]
    predicted = predicted[filt]
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    mse = np.mean((predicted - trues)**2)
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    r2 = max(0, 1 - np.sum((trues-predicted)**2) / np.sum((trues - np.mean(trues))**2))
    return corr, mae, mse, rae, rmse, r2

In [8]:
def eval_quantiles(lower, upper, trues, preds):
    N = len(trues)
    icp = 1.0*np.sum((trues>lower) & (trues<upper)) / N
    diffs = np.maximum(0, upper-lower)
    mil = np.sum(diffs) / N
    rmil = 0.0
    for i in xrange(N):
        if trues[i] != preds[i]:
            rmil += diffs[i] / (np.abs(trues[i]-preds[i]))
    rmil = rmil / N
    clc = np.exp(-rmil*(icp-0.95))
    return icp, mil, rmil, clc

## Linear regression baseline

In [9]:
# place id of interest
ix = 7
print ids[ix]

Gc


In [10]:
# true values we want to predict
y_true = y_test[:,0,ix,0,0] * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]

# naive baseline
preds_naive = y[n_train-STEPS_AHEAD:n_test-STEPS_AHEAD,0,ix,0,0] * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]
corr, mae_naive, mse, rae, rmse_naive, r2_naive = compute_error(y_true, preds_naive)
print "NAIVE: %.3f   %.3f   %.3f" % (mae_naive,rmse_naive,r2_naive)

# linear regression
regr = linear_model.LinearRegression()
regr.fit(X_train[:,:,ix,0,0], y_train[:,0,ix,0,0])
preds_lr = regr.predict(X_test[:,:,ix,0,0]) * removed_std_all[ix][n_train:n_test] + removed_seasonality_all[ix][n_train:n_test]

corr, mae_lr, mse, rae, rmse_lr, r2_lr = compute_error(y_true, preds_lr)
print "LR:    %.3f   %.3f   %.3f" % (mae_lr,rmse_lr,r2_lr)

NAIVE: 2.826   4.891   0.730
LR:    2.321   4.229   0.798


## Specify model

In [11]:
def tilted_loss(q,y,f):
    e = (y-f)
    # The term inside k.mean is a one line simplification of the first equation
    return K.mean(q*e + K.clip(-e, K.epsilon(), np.inf), axis=-1)

In [12]:
def build_model(loss="mse", num_outs=1):
    model = Sequential()

    #model.add(LSTM(layers[1], input_shape=(None, layers[0]), return_sequences=True))
    model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3),
                   input_shape=(None, 9, 1, 1),
                   padding='same', return_sequences=True))
    model.add(Dropout(0.2))

    model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3),
                       padding='same', return_sequences=False))
    
    #model.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
    #           activation='linear',
    #           padding='same', data_format='channels_last'))

    #model.add(LSTM(layers[2], return_sequences=False))
    model.add(Dropout(0.2))
    
    #model.add(Dense(units=1, kernel_regularizer=regularizers.l2(0.00001)))
    model.add(Dense(units=num_outs))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss=loss, optimizer="rmsprop")
    
    return model

## Train model and test

In [13]:
print X_train.shape
print X_train[:,:,:,0,0].transpose([0,2,1]).shape
print y_train.shape

(25632, 50, 9, 1, 1)
(25632, 9, 50)
(25632, 1, 9, 1, 1)


In [14]:
model = build_model(num_outs=1)

# checkpoint best model
checkpoint = ModelCheckpoint("convlstm_mean.best.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

model.fit(
    X_train,
    y_train[:,:,:,:,0].swapaxes(1,2),
    batch_size=512,
    epochs=100,
    validation_split=0.2,
    callbacks=[checkpoint],
    verbose=2)  

Train on 20505 samples, validate on 5127 samples
Epoch 1/100
Epoch 00000: val_loss improved from inf to 0.65947, saving model to convlstm_mean.best.hdf5
13s - loss: 0.9740 - val_loss: 0.6595
Epoch 2/100
Epoch 00001: val_loss improved from 0.65947 to 0.41650, saving model to convlstm_mean.best.hdf5
12s - loss: 0.6426 - val_loss: 0.4165
Epoch 3/100
Epoch 00002: val_loss improved from 0.41650 to 0.33184, saving model to convlstm_mean.best.hdf5
12s - loss: 0.4902 - val_loss: 0.3318
Epoch 4/100
Epoch 00003: val_loss improved from 0.33184 to 0.30196, saving model to convlstm_mean.best.hdf5
12s - loss: 0.4142 - val_loss: 0.3020
Epoch 5/100
Epoch 00004: val_loss improved from 0.30196 to 0.27038, saving model to convlstm_mean.best.hdf5
12s - loss: 0.3643 - val_loss: 0.2704
Epoch 6/100
Epoch 00005: val_loss improved from 0.27038 to 0.25854, saving model to convlstm_mean.best.hdf5
13s - loss: 0.3402 - val_loss: 0.2585
Epoch 7/100
Epoch 00006: val_loss improved from 0.25854 to 0.24603, saving mode

Epoch 00084: val_loss did not improve
13s - loss: 0.2797 - val_loss: 0.2442
Epoch 86/100
Epoch 00085: val_loss did not improve
13s - loss: 0.2808 - val_loss: 0.2323
Epoch 87/100
Epoch 00086: val_loss did not improve
13s - loss: 0.2798 - val_loss: 0.2313
Epoch 88/100
Epoch 00087: val_loss did not improve
12s - loss: 0.2803 - val_loss: 0.2356
Epoch 89/100
Epoch 00088: val_loss did not improve
13s - loss: 0.2801 - val_loss: 0.2367
Epoch 90/100
Epoch 00089: val_loss did not improve
12s - loss: 0.2806 - val_loss: 0.2373
Epoch 91/100
Epoch 00090: val_loss did not improve
13s - loss: 0.2803 - val_loss: 0.2299
Epoch 92/100
Epoch 00091: val_loss did not improve
13s - loss: 0.2798 - val_loss: 0.2375
Epoch 93/100
Epoch 00092: val_loss did not improve
13s - loss: 0.2781 - val_loss: 0.2303
Epoch 94/100
Epoch 00093: val_loss did not improve
13s - loss: 0.2781 - val_loss: 0.2309
Epoch 95/100
Epoch 00094: val_loss did not improve
12s - loss: 0.2793 - val_loss: 0.2355
Epoch 96/100
Epoch 00095: val_loss

In [15]:
# load weights
model.load_weights("convlstm_mean.best.hdf5")

# make predictions
predictions = model.predict(X_test)
print predictions.shape

(25920, 9, 1, 1)


## Train independent NNs for the quantiles

In [16]:
quantiles = [0.05, 0.10, 0.90, 0.95]

In [17]:
models_q = []
for q in quantiles:
    print "training NN for quantile:", q
    
    # checkpoint best model
    checkpoint = ModelCheckpoint("convlstm_q%2f.best.hdf5" % (q,), monitor='val_loss', verbose=0, save_best_only=True, mode='min')

    # fit model to the 5% quantile
    model_q = build_model(loss=lambda y,f: tilted_loss(q,y,f))
    model_q.fit(
        X_train,
        y_train[:,:,:,:,0].swapaxes(1,2),
        batch_size=512,
        epochs=100,
        validation_split=0.2,
        callbacks=[checkpoint],
        verbose=2)  
    models_q.append(model_q)

training NN for quantile: 0.05
Train on 20505 samples, validate on 5127 samples
Epoch 1/100
13s - loss: 0.1523 - val_loss: 0.1006
Epoch 2/100
13s - loss: 0.1017 - val_loss: 0.0926
Epoch 3/100
13s - loss: 0.0906 - val_loss: 0.0987
Epoch 4/100
12s - loss: 0.0829 - val_loss: 0.0711
Epoch 5/100
13s - loss: 0.0762 - val_loss: 0.0660
Epoch 6/100
12s - loss: 0.0701 - val_loss: 0.0610
Epoch 7/100
12s - loss: 0.0656 - val_loss: 0.0580
Epoch 8/100
13s - loss: 0.0623 - val_loss: 0.0522
Epoch 9/100
12s - loss: 0.0591 - val_loss: 0.0518
Epoch 10/100
13s - loss: 0.0566 - val_loss: 0.0499
Epoch 11/100
13s - loss: 0.0546 - val_loss: 0.0432
Epoch 12/100
12s - loss: 0.0526 - val_loss: 0.0420
Epoch 13/100
13s - loss: 0.0512 - val_loss: 0.0417
Epoch 14/100
13s - loss: 0.0501 - val_loss: 0.0435
Epoch 15/100
12s - loss: 0.0495 - val_loss: 0.0406
Epoch 16/100
13s - loss: 0.0489 - val_loss: 0.0400
Epoch 17/100
12s - loss: 0.0483 - val_loss: 0.0399
Epoch 18/100
13s - loss: 0.0479 - val_loss: 0.0429
Epoch 19/10

13s - loss: 0.0677 - val_loss: 0.0595
Epoch 60/100
13s - loss: 0.0679 - val_loss: 0.0586
Epoch 61/100
12s - loss: 0.0678 - val_loss: 0.0590
Epoch 62/100
13s - loss: 0.0674 - val_loss: 0.0593
Epoch 63/100
13s - loss: 0.0676 - val_loss: 0.0623
Epoch 64/100
13s - loss: 0.0677 - val_loss: 0.0578
Epoch 65/100
13s - loss: 0.0675 - val_loss: 0.0580
Epoch 66/100
13s - loss: 0.0673 - val_loss: 0.0583
Epoch 67/100
12s - loss: 0.0672 - val_loss: 0.0587
Epoch 68/100
13s - loss: 0.0672 - val_loss: 0.0583
Epoch 69/100
12s - loss: 0.0673 - val_loss: 0.0573
Epoch 70/100
12s - loss: 0.0671 - val_loss: 0.0583
Epoch 71/100
13s - loss: 0.0672 - val_loss: 0.0608
Epoch 72/100
13s - loss: 0.0671 - val_loss: 0.0594
Epoch 73/100
13s - loss: 0.0671 - val_loss: 0.0599
Epoch 74/100
13s - loss: 0.0671 - val_loss: 0.0616
Epoch 75/100
13s - loss: 0.0670 - val_loss: 0.0637
Epoch 76/100
13s - loss: 0.0672 - val_loss: 0.0580
Epoch 77/100
13s - loss: 0.0670 - val_loss: 0.0570
Epoch 78/100
13s - loss: 0.0668 - val_loss: 

13s - loss: 0.0485 - val_loss: 0.0422
Epoch 18/100
13s - loss: 0.0480 - val_loss: 0.0411
Epoch 19/100
13s - loss: 0.0477 - val_loss: 0.0432
Epoch 20/100
13s - loss: 0.0475 - val_loss: 0.0411
Epoch 21/100
13s - loss: 0.0473 - val_loss: 0.0447
Epoch 22/100
13s - loss: 0.0467 - val_loss: 0.0415
Epoch 23/100
13s - loss: 0.0463 - val_loss: 0.0415
Epoch 24/100
13s - loss: 0.0464 - val_loss: 0.0409
Epoch 25/100
13s - loss: 0.0462 - val_loss: 0.0408
Epoch 26/100
13s - loss: 0.0460 - val_loss: 0.0403
Epoch 27/100
13s - loss: 0.0458 - val_loss: 0.0418
Epoch 28/100
13s - loss: 0.0458 - val_loss: 0.0426
Epoch 29/100
12s - loss: 0.0456 - val_loss: 0.0408
Epoch 30/100
13s - loss: 0.0452 - val_loss: 0.0399
Epoch 31/100
13s - loss: 0.0453 - val_loss: 0.0408
Epoch 32/100
12s - loss: 0.0450 - val_loss: 0.0404
Epoch 33/100
13s - loss: 0.0450 - val_loss: 0.0398
Epoch 34/100
13s - loss: 0.0450 - val_loss: 0.0406
Epoch 35/100
13s - loss: 0.0449 - val_loss: 0.0401
Epoch 36/100
13s - loss: 0.0448 - val_loss: 

In [18]:
predictions_q = []
for i in xrange(len(quantiles)):
    q = quantiles[i]
    print "making predictions for quantile:", q
    
    # load weights
    models_q[i].load_weights("convlstm_q%2f.best.hdf5" % (q,))

    # make predictions
    predictions_q.append(models_q[i].predict(X_test))

making predictions for quantile: 0.05
making predictions for quantile: 0.1
making predictions for quantile: 0.9
making predictions for quantile: 0.95


## Linear Quantile Regression baselines

In [19]:
print X_train.shape
print y_train.shape

(25632, 50, 9, 1, 1)
(25632, 1, 9, 1, 1)


In [20]:
import warnings
warnings.filterwarnings("ignore")

col_names = ["l%d" % (i,) for i in xrange(NUM_LAGS-STEPS_AHEAD+1)]
col_names.append("y")
qr_str = "y ~ " + ''.join(["+l%d" % (x,) for x in range(NUM_LAGS-STEPS_AHEAD+1)])[1:]

predictions_lr = []
predictions_q_lr = []
for k in xrange(len(quantiles)):
    q = quantiles[k]
    print "running quantile regression for quantile:", q
    
    preds_lr = np.zeros((n_test-n_train, n_places))
    preds_qr = np.zeros((n_test-n_train, n_places))
    for i in xrange(n_places):
        regr = linear_model.LinearRegression()
        regr.fit(X_train[:,:,i,0,0], y_train[:,0,ix,0,0])
        preds_lr[:,i] = regr.predict(X_test[:,:,i,0,0]) * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

        data = pd.DataFrame(data = np.hstack([X_train[:,:,i,0,0], y_train[:,0,ix,0,:]]), columns = col_names)
        data_test = pd.DataFrame(data = np.hstack([X_test[:,:,i,0,0], y_test[:,0,ix,0,:]]), columns = col_names)

        #qr = smf.quantreg('y ~ l0+l1+l2+l3+l4+l5+l6+l7+l8', data)
        qr = smf.quantreg(qr_str, data)
        res = qr.fit(q=q)
        preds_qr[:,i] = res.predict(data_test) * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    
    predictions_lr.append(preds_lr)
    predictions_q_lr.append(preds_qr)

running quantile regression for quantile: 0.05
running quantile regression for quantile: 0.1
running quantile regression for quantile: 0.9
running quantile regression for quantile: 0.95


## Simultaneous Quantiles NN

In [21]:
def multi_tilted_loss(quantiles,y,f):
    print y
    print f
    # The term inside k.mean is a one line simplification of the first equation
    loss = K.mean(K.square(y[:,:,:,0]-f[:,:,:,0]), axis=-1)
    for k in xrange(len(quantiles)):
        q = quantiles[k]
        e = (y[:,:,:,k+1]-f[:,:,:,k+1])
        loss += K.mean(q*e + K.clip(-e, K.epsilon(), np.inf), axis=-1)
    return loss

In [22]:
global previous_losses
previous_losses = [1.0,1.0,1.0,1.0,1.0]
def multi_tilted_loss_weighted(quantiles,y,f):
    global previous_losses
    print y
    print f
    max_prev_loss = tf.reduce_max(previous_losses)
    
    # The term inside k.mean is a one line simplification of the first equation
    losses = [(max_prev_loss/previous_losses[0]) * K.mean(K.square(y[:,:,:,0]-f[:,:,:,0]), axis=-1)]
    for k in xrange(len(quantiles)):
        q = quantiles[k]
        e = (y[:,:,:,k+1]-f[:,:,:,k+1])
        losses.append((max_prev_loss/previous_losses[k+1]) * K.mean(q*e + K.clip(-e, K.epsilon(), np.inf), axis=-1))
    previous_losses = losses
    loss = tf.reduce_sum(losses)
    return loss

In [23]:
print y_train[:,:,:,:,0].swapaxes(1,2).shape

(25632, 9, 1, 1)


In [24]:
y_traink = y_train[:,:,:,:,0].swapaxes(1,2)
for k in xrange(len(quantiles)):
    y_traink = np.concatenate((y_traink, y_train[:,:,:,:,0].swapaxes(1,2)), axis=3)
print y_traink.shape

(25632, 9, 1, 5)


In [25]:
# checkpoint best model
checkpoint = ModelCheckpoint("convlstm_multi.best.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# fit model to the 95% quantile
model_multi = build_model(loss=lambda y,f: multi_tilted_loss_weighted(quantiles,y,f), num_outs=1+len(quantiles))
model_multi.fit(
    X_train,
    y_traink,
    batch_size=128,
    epochs=80,
    validation_split=0.2,
    callbacks=[checkpoint],
    verbose=2)

Tensor("activation_6_target:0", shape=(?, ?, ?, ?), dtype=float32)
Tensor("activation_6/Identity:0", shape=(?, 9, 1, 5), dtype=float32)
Train on 20505 samples, validate on 5127 samples
Epoch 1/80
Epoch 00000: val_loss improved from inf to 855.00848, saving model to convlstm_multi.best.hdf5
52s - loss: 1622.3801 - val_loss: 855.0085
Epoch 2/80
Epoch 00001: val_loss improved from 855.00848 to 596.50465, saving model to convlstm_multi.best.hdf5
51s - loss: 912.6572 - val_loss: 596.5046
Epoch 3/80
Epoch 00002: val_loss improved from 596.50465 to 540.63502, saving model to convlstm_multi.best.hdf5
51s - loss: 738.7509 - val_loss: 540.6350
Epoch 4/80
Epoch 00003: val_loss improved from 540.63502 to 527.20755, saving model to convlstm_multi.best.hdf5
51s - loss: 691.0073 - val_loss: 527.2075
Epoch 5/80
Epoch 00004: val_loss improved from 527.20755 to 523.02327, saving model to convlstm_multi.best.hdf5
52s - loss: 669.5314 - val_loss: 523.0233
Epoch 6/80
Epoch 00005: val_loss improved from 523

Epoch 00070: val_loss did not improve
51s - loss: 577.6706 - val_loss: 485.9359
Epoch 72/80
Epoch 00071: val_loss did not improve
52s - loss: 576.3670 - val_loss: 485.3061
Epoch 73/80
Epoch 00072: val_loss did not improve
52s - loss: 574.9024 - val_loss: 484.1358
Epoch 74/80
Epoch 00073: val_loss did not improve
52s - loss: 575.7092 - val_loss: 484.6609
Epoch 75/80
Epoch 00074: val_loss did not improve
52s - loss: 576.0999 - val_loss: 484.8317
Epoch 76/80
Epoch 00075: val_loss did not improve
52s - loss: 575.4316 - val_loss: 490.9767
Epoch 77/80
Epoch 00076: val_loss did not improve
52s - loss: 574.4390 - val_loss: 483.2464
Epoch 78/80
Epoch 00077: val_loss did not improve
52s - loss: 571.1588 - val_loss: 487.0911
Epoch 79/80
Epoch 00078: val_loss did not improve
52s - loss: 575.2909 - val_loss: 484.7599
Epoch 80/80
Epoch 00079: val_loss did not improve
51s - loss: 574.4791 - val_loss: 485.1796


In [26]:
a = 1

In [27]:
# load weights
model_multi.load_weights("convlstm_multi.best.hdf5")

# make predictions
predictions_multi = model_multi.predict(X_test)
print predictions_multi.shape

(25920, 9, 1, 5)


## Compare results for different methods

In [28]:
predictions_multi.shape

(25920, 9, 1, 5)

In [29]:
def compare_methods(i):
    trues = y_test[:,0,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    preds_qr = predictions_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr1 = predictions_q_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr2 = predictions_q_lr[1][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr1 = predictions_q_lr[2][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr2 = predictions_q_lr[3][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]

    preds_dl = predictions[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl1 = predictions_q[0][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl2 = predictions_q[1][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl1 = predictions_q[2][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl2 = predictions_q[3][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    preds_multi = predictions_multi[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi1 = predictions_multi[:,i,0,1] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi2 = predictions_multi[:,i,0,2] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi1 = predictions_multi[:,i,0,3] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi2 = predictions_multi[:,i,0,4] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    # evaluate prediction means
    print "Prediction mean:"
    print "\tMAE\tRMSE\tR2"
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_qr)
    print "LR: \t%.3f\t%.3f\t%.3f" % (mae,rmse,r2)
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_dl)
    print "DLQR: \t%.3f\t%.3f\t%.3f" % (mae,rmse,r2)
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_multi)
    print "MULTI: \t%.3f\t%.3f\t%.3f" % (mae,rmse,r2)

    # evaluate quantiles
    print "\nQuantiles 90%:"
    print "\tICP\tMIL\tRMIL"
    icp, mil, rmil, clc = eval_quantiles(lower_qr1, upper_qr2, trues, preds_qr)
    print "QR: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_dl1, upper_dl2, trues, preds_dl)
    print "DLQR: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_multi1, upper_multi2, trues, preds_multi)
    print "MULTI: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)

    print "\nQuantiles 60%:"
    print "\tICP\tMIL\tRMIL"
    icp, mil, rmil, clc = eval_quantiles(lower_qr2, upper_qr1, trues, preds_qr)
    print "QR: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_dl2, upper_dl1, trues, preds_dl)
    print "DLQR: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_multi2, upper_multi1, trues, preds_multi)
    print "MULTI: \t%.3f\t%.3f\t%.3f" % (icp, mil, rmil)

In [30]:
compare_methods(ix)

Prediction mean:
	MAE	RMSE	R2
LR: 	2.321	4.229	0.798
DLQR: 	1.955	3.828	0.835
MULTI: 	1.929	3.817	0.835

Quantiles 90%:
	ICP	MIL	RMIL
QR: 	0.910	10.719	168.890
DLQR: 	0.891	7.959	56.919
MULTI: 	0.932	8.701	73.288

Quantiles 60%:
	ICP	MIL	RMIL
QR: 	0.817	6.275	97.771
DLQR: 	0.813	6.010	45.891
MULTI: 	0.852	6.319	54.228


In [31]:
maes_lr = []
rmses_lr = []
r2s_lr = []
maes_dl = []
rmses_dl = []
r2s_dl = []
maes_multi = []
rmses_multi = []
r2s_multi = []
for i in xrange(n_places):
    trues = y_test[:,0,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    preds_qr = predictions_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr1 = predictions_q_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr2 = predictions_q_lr[1][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr1 = predictions_q_lr[2][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr2 = predictions_q_lr[3][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]

    preds_dl = predictions[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl1 = predictions_q[0][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl2 = predictions_q[1][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl1 = predictions_q[2][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl2 = predictions_q[3][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    preds_multi = predictions_multi[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi1 = predictions_multi[:,i,0,1] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi2 = predictions_multi[:,i,0,2] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi1 = predictions_multi[:,i,0,3] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi2 = predictions_multi[:,i,0,4] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    # evaluate prediction means
    s = "[%d]\tMean:" % (i,)
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_qr)
    maes_lr.append(mae)
    rmses_lr.append(rmse)
    r2s_lr.append(r2)
    s += "\t(QR: %.2f %.2f %.2f)" % (mae,rmse,r2)
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_dl)
    maes_dl.append(mae)
    rmses_dl.append(rmse)
    r2s_dl.append(r2)
    s += "\t(DLQR: %.2f %.2f %.2f)  " % (mae,rmse,r2)
    corr, mae, mse, rae, rmse, r2 = compute_error(trues, preds_multi)
    maes_multi.append(mae)
    rmses_multi.append(rmse)
    r2s_multi.append(r2)
    s += "\t(MULTI: %.2f %.2f %.2f)" % (mae,rmse,r2)
    print s

[0]	Mean:	(QR: 3.04 4.94 0.70)	(DLQR: 1.99 3.99 0.80)  	(MULTI: 1.97 4.00 0.80)
[1]	Mean:	(QR: 3.17 4.94 0.60)	(DLQR: 1.74 3.47 0.80)  	(MULTI: 1.73 3.48 0.80)
[2]	Mean:	(QR: 3.32 5.11 0.66)	(DLQR: 1.92 3.70 0.82)  	(MULTI: 1.91 3.71 0.82)
[3]	Mean:	(QR: 3.38 5.22 0.71)	(DLQR: 1.94 3.73 0.85)  	(MULTI: 1.91 3.73 0.85)
[4]	Mean:	(QR: 3.26 5.13 0.70)	(DLQR: 1.99 3.88 0.83)  	(MULTI: 1.98 3.87 0.83)
[5]	Mean:	(QR: 3.08 4.92 0.68)	(DLQR: 1.94 3.76 0.81)  	(MULTI: 1.92 3.78 0.81)
[6]	Mean:	(QR: 3.29 5.20 0.69)	(DLQR: 1.96 3.86 0.83)  	(MULTI: 1.94 3.85 0.83)
[7]	Mean:	(QR: 2.32 4.23 0.80)	(DLQR: 1.96 3.83 0.83)  	(MULTI: 1.93 3.82 0.84)
[8]	Mean:	(QR: 3.00 4.83 0.73)	(DLQR: 1.99 3.88 0.83)  	(MULTI: 1.99 3.88 0.83)


In [32]:
print "ERRORS:"
print "LR:    %.3f %.3f %.3f" % (np.mean(maes_lr),np.mean(rmses_lr),np.mean(r2s_lr)) 
print "DL:    %.3f %.3f %.3f" % (np.mean(maes_dl),np.mean(rmses_dl),np.mean(r2s_dl)) 
print "MULTI: %.3f %.3f %.3f" % (np.mean(maes_multi),np.mean(rmses_multi),np.mean(r2s_multi)) 

ERRORS:
LR:    3.097 4.947 0.695
DL:    1.937 3.788 0.823
MULTI: 1.921 3.790 0.823


In [33]:
icps_qr = []
mils_qr = []
rmils_qr = []
icps_dl = []
mils_dl = []
rmils_dl = []
icps_multi = []
mils_multi = []
rmils_multi = []
for i in xrange(n_places):
    trues = y_test[:,0,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    preds_qr = predictions_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr1 = predictions_q_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr2 = predictions_q_lr[1][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr1 = predictions_q_lr[2][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr2 = predictions_q_lr[3][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]

    preds_dl = predictions[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl1 = predictions_q[0][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl2 = predictions_q[1][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl1 = predictions_q[2][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl2 = predictions_q[3][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    preds_multi = predictions_multi[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi1 = predictions_multi[:,i,0,1] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi2 = predictions_multi[:,i,0,2] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi1 = predictions_multi[:,i,0,3] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi2 = predictions_multi[:,i,0,4] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]


    # evaluate quantiles
    s = "[%d]\tQuantiles 95:" % (i,)
    icp, mil, rmil, clc = eval_quantiles(lower_qr1, upper_qr2, trues, preds_qr)
    icps_qr.append(icp)
    mils_qr.append(mil)
    rmils_qr.append(rmil)
    s += "\t(QR: %.2f %.2f %.2f)" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_dl1, upper_dl2, trues, preds_dl)
    icps_dl.append(icp)
    mils_dl.append(mil)
    rmils_dl.append(rmil)
    s += "\t(DLQR: %.2f %.2f %.2f)  " % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_multi1, upper_multi2, trues, preds_multi)
    icps_multi.append(icp)
    mils_multi.append(mil)
    rmils_multi.append(rmil)
    s += "\t(MULTI: %.2f %.2f %.2f)" % (icp, mil, rmil)
    print s

[0]	Quantiles 95:	(QR: 0.94 19.12 49.92)	(DLQR: 0.90 8.03 28.78)  	(MULTI: 0.93 8.67 42.75)
[1]	Quantiles 95:	(QR: 0.95 20.59 57.67)	(DLQR: 0.89 7.04 31.16)  	(MULTI: 0.93 7.70 30.63)
[2]	Quantiles 95:	(QR: 0.95 21.32 68.04)	(DLQR: 0.89 7.83 28.68)  	(MULTI: 0.93 8.50 49.69)
[3]	Quantiles 95:	(QR: 0.95 22.88 83.00)	(DLQR: 0.89 7.99 36.80)  	(MULTI: 0.94 8.74 45.00)
[4]	Quantiles 95:	(QR: 0.95 21.91 113.76)	(DLQR: 0.89 8.15 223.61)  	(MULTI: 0.93 8.70 67.33)
[5]	Quantiles 95:	(QR: 0.95 20.50 108.27)	(DLQR: 0.88 7.90 28.19)  	(MULTI: 0.93 8.58 36.64)
[6]	Quantiles 95:	(QR: 0.95 22.27 92.70)	(DLQR: 0.89 7.99 170.10)  	(MULTI: 0.93 8.61 49.06)
[7]	Quantiles 95:	(QR: 0.91 10.72 168.89)	(DLQR: 0.89 7.96 56.92)  	(MULTI: 0.93 8.70 73.29)
[8]	Quantiles 95:	(QR: 0.95 19.76 60.45)	(DLQR: 0.90 8.41 26.36)  	(MULTI: 0.92 8.68 37.80)


In [34]:
print "QUANTILES 95 (MEAN):"
print "QR:    %.3f %.3f %.3f" % (np.mean(icps_qr),np.mean(mils_qr),np.mean(rmils_qr)) 
print "DLQR:  %.3f %.3f %.3f" % (np.mean(icps_dl),np.mean(mils_dl),np.mean(rmils_dl)) 
print "MULTI: %.3f %.3f %.3f" % (np.mean(icps_multi),np.mean(mils_multi),np.mean(rmils_multi)) 

QUANTILES 95 (MEAN):
QR:    0.943 19.897 89.190
DLQR:  0.891 7.922 70.067
MULTI: 0.930 8.543 48.021


In [35]:
print "QUANTILES 90 (MEDIAN):"
print "QR:    %.3f %.3f %.3f" % (np.median(icps_qr),np.median(mils_qr),np.median(rmils_qr)) 
print "DLQR:  %.3f %.3f %.3f" % (np.median(icps_dl),np.median(mils_dl),np.median(rmils_dl)) 
print "MULTI: %.3f %.3f %.3f" % (np.median(icps_multi),np.median(mils_multi),np.median(rmils_multi)) 

QUANTILES 90 (MEDIAN):
QR:    0.947 20.586 83.000
DLQR:  0.889 7.985 31.163
MULTI: 0.929 8.666 44.998


In [36]:
icps_qr = []
mils_qr = []
rmils_qr = []
icps_dl = []
mils_dl = []
rmils_dl = []
icps_multi = []
mils_multi = []
rmils_multi = []
for i in xrange(n_places):
    trues = y_test[:,0,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    preds_qr = predictions_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr1 = predictions_q_lr[0][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    lower_qr2 = predictions_q_lr[1][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr1 = predictions_q_lr[2][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]
    upper_qr2 = predictions_q_lr[3][:,i] #* matrix_std[i,j] + matrix_trend[n_train:n_test,i,j]

    preds_dl = predictions[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl1 = predictions_q[0][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_dl2 = predictions_q[1][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl1 = predictions_q[2][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_dl2 = predictions_q[3][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]

    preds_multi = predictions_multi[:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi1 = predictions_multi[:,i,0,1] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    lower_multi2 = predictions_multi[:,i,0,2] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi1 = predictions_multi[:,i,0,3] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
    upper_multi2 = predictions_multi[:,i,0,4] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]


    # evaluate quantiles
    s = "[%d]\tQuantiles 60:" % (i,)
    icp, mil, rmil, clc = eval_quantiles(lower_qr2, upper_qr1, trues, preds_qr)
    icps_qr.append(icp)
    mils_qr.append(mil)
    rmils_qr.append(rmil)
    s += "\t(QR: %.2f %.2f %.2f)" % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_dl2, upper_dl1, trues, preds_dl)
    icps_dl.append(icp)
    mils_dl.append(mil)
    rmils_dl.append(rmil)
    s += "\t(DLQR: %.2f %.2f %.2f)  " % (icp, mil, rmil)
    icp, mil, rmil, clc = eval_quantiles(lower_multi2, upper_multi1, trues, preds_multi)
    icps_multi.append(icp)
    mils_multi.append(mil)
    rmils_multi.append(rmil)
    s += "\t(MULTI: %.2f %.2f %.2f)" % (icp, mil, rmil)
    print s

[0]	Quantiles 60:	(QR: 0.88 11.33 29.67)	(DLQR: 0.81 6.01 21.32)  	(MULTI: 0.85 6.30 31.44)
[1]	Quantiles 60:	(QR: 0.89 12.94 36.33)	(DLQR: 0.81 5.33 24.32)  	(MULTI: 0.85 5.59 22.73)
[2]	Quantiles 60:	(QR: 0.88 13.20 41.95)	(DLQR: 0.81 5.97 22.37)  	(MULTI: 0.84 6.17 36.91)
[3]	Quantiles 60:	(QR: 0.90 14.11 50.89)	(DLQR: 0.82 6.06 28.68)  	(MULTI: 0.86 6.35 33.56)
[4]	Quantiles 60:	(QR: 0.89 13.24 68.91)	(DLQR: 0.81 6.11 176.46)  	(MULTI: 0.84 6.32 49.53)
[5]	Quantiles 60:	(QR: 0.89 12.22 64.45)	(DLQR: 0.81 6.02 21.78)  	(MULTI: 0.85 6.22 27.17)
[6]	Quantiles 60:	(QR: 0.90 13.51 55.20)	(DLQR: 0.82 6.07 120.13)  	(MULTI: 0.85 6.25 35.97)
[7]	Quantiles 60:	(QR: 0.82 6.28 97.77)	(DLQR: 0.81 6.01 45.89)  	(MULTI: 0.85 6.32 54.23)
[8]	Quantiles 60:	(QR: 0.89 11.77 36.23)	(DLQR: 0.83 6.23 20.02)  	(MULTI: 0.84 6.29 28.11)


In [37]:
print "QUANTILES 60:"
print "QR:    %.3f %.3f %.3f" % (np.mean(icps_qr),np.mean(mils_qr),np.mean(rmils_qr)) 
print "DLQR:  %.3f %.3f %.3f" % (np.mean(icps_dl),np.mean(mils_dl),np.mean(rmils_dl)) 
print "MULTI: %.3f %.3f %.3f" % (np.mean(icps_multi),np.mean(mils_multi),np.mean(rmils_multi)) 

QUANTILES 60:
QR:    0.882 12.065 53.488
DLQR:  0.815 5.980 53.441
MULTI: 0.847 6.202 35.516


In [38]:
print "QUANTILES 60:"
print "QR:    %.3f %.3f %.3f" % (np.median(icps_qr),np.median(mils_qr),np.median(rmils_qr)) 
print "DLQR:  %.3f %.3f %.3f" % (np.median(icps_dl),np.median(mils_dl),np.median(rmils_dl)) 
print "MULTI: %.3f %.3f %.3f" % (np.median(icps_multi),np.median(mils_multi),np.median(rmils_multi)) 

QUANTILES 60:
QR:    0.893 12.940 50.885
DLQR:  0.813 6.023 24.319
MULTI: 0.848 6.295 33.561


In [39]:
loss_qr = 0.0
loss_dl = 0.0
loss_multi = 0.0
for i in xrange(n_places):
     for k in xrange(len(quantiles)-1):
        q1 = predictions_q_lr[k][:,i]
        q2 = predictions_q_lr[k+1][:,i]
        loss_qr += np.sum(np.maximum(0.0, q1 - q2)) / len(q1)

        q1 = predictions_q[k][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
        q2 = predictions_q[k+1][:,i,0,0] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
        loss_dl += np.sum(np.maximum(0.0, q1 - q2)) / len(q1)

        q1 = predictions_multi[:,i,0,1+k] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
        q2 = predictions_multi[:,i,0,1+k+1] * removed_std_all[i][n_train:n_test] + removed_seasonality_all[i][n_train:n_test]
        loss_multi += np.sum(np.maximum(0.0, q1 - q2)) / len(q1)

print "cross loss qr:", loss_qr
print "cross loss dl:", loss_dl
print "cross loss multi:", loss_multi

cross loss qr: 0.0303701671556
cross loss dl: 0.56815735353
cross loss multi: 0.0


In [40]:
for k in xrange(len(quantiles)-1):
    loss_qr = np.sum(predictions_q_lr[k][:,ix] > predictions_q_lr[k+1][:,ix])
    loss_dl = np.sum(predictions_q[k][:,ix,0,0] > predictions_q[k+1][:,ix,0,0])
    loss_multi = np.sum(predictions_multi[:,ix,0,1+k] > predictions_multi[:,ix,0,1+k+1])
    print "[%d-%d] crossing cases: QR: %d\tDL: %d\t  MULTI: %d" % (k,k+1,loss_qr,loss_dl,loss_multi)

[0-1] crossing cases: QR: 71	DL: 2847	  MULTI: 0
[1-2] crossing cases: QR: 0	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 154	DL: 1435	  MULTI: 0


In [41]:
sum_qr = 0
sum_dl = 0
sum_multi = 0
for i in xrange(n_places):
    print "Road segment [%d]" % (i,)
    for k in xrange(len(quantiles)-1):
        loss_qr = np.sum(predictions_q_lr[k][:,i] > predictions_q_lr[k+1][:,i])
        sum_qr += loss_qr
        loss_dl = np.sum(predictions_q[k][:,i,0,0] > predictions_q[k+1][:,i,0,0])
        sum_dl += loss_dl
        loss_multi = np.sum(predictions_multi[:,i,0,1+k] > predictions_multi[:,i,0,1+k+1])
        sum_multi += loss_multi
        print "[%d-%d] crossing cases: QR: %d\tDL: %d\t  MULTI: %d" % (k,k+1,loss_qr,loss_dl,loss_multi)
print "sum_qr:", sum_qr
print "sum_dl:", sum_dl
print "sum_multi:", sum_multi

Road segment [0]
[0-1] crossing cases: QR: 53	DL: 5854	  MULTI: 0
[1-2] crossing cases: QR: 1	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 69	DL: 83	  MULTI: 0
Road segment [1]
[0-1] crossing cases: QR: 13	DL: 4604	  MULTI: 0
[1-2] crossing cases: QR: 0	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 14	DL: 280	  MULTI: 0
Road segment [2]
[0-1] crossing cases: QR: 33	DL: 3687	  MULTI: 0
[1-2] crossing cases: QR: 16	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 24	DL: 644	  MULTI: 0
Road segment [3]
[0-1] crossing cases: QR: 32	DL: 4240	  MULTI: 0
[1-2] crossing cases: QR: 0	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 22	DL: 1398	  MULTI: 0
Road segment [4]
[0-1] crossing cases: QR: 19	DL: 3681	  MULTI: 0
[1-2] crossing cases: QR: 0	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 103	DL: 1694	  MULTI: 0
Road segment [5]
[0-1] crossing cases: QR: 25	DL: 3681	  MULTI: 0
[1-2] crossing cases: QR: 0	DL: 0	  MULTI: 0
[2-3] crossing cases: QR: 85	DL: 2028	  MULTI: 0
Road segment [6]
[0-1] crossing cases: QR: